Here, we apply gradient descent for an SDP problem.

We set up the problem identically to that in Problem 9 in HW1 and HW2. 

In [11]:
import numpy as np

# Construct the anchor points
a1 = np.array([1.0, 0.0])
a2 = np.array([-1.0, 0.0])
a3 = np.array([0.0, 2.0])


# Construct the vectors of our triangle
v1 = a1 - a2
v2 = a3 - a2

# Function to generate a random point inside or outside of 
# our convex hull. 
def point_generator(outside=False):
    # Compute a random point within the quadrilateral
    rand_point = a2 + (np.random.random() * v1 + np.random.random() * v2)
    
    # Compute the barycentric coordinates to determine whether the point is inside
    # the triangle, or outside of it
    alpha = ((a2[1] - a3[1])*(rand_point[0] - a3[0]) + (a3[0] - a2[0])* \
            (rand_point[1] - a3[1])) / ((a2[1] - a3[1])*(a1[0] - a3[0]) + \
            (a3[0] - a2[0])*(a1[1] - a3[1]))
    beta = ((a3[1] - a1[1])*(rand_point[0] - a3[0]) + (a1[0] - a3[0])* \
            (rand_point[1] - a3[1])) / ((a2[1] - a3[1])*(a1[0] - a3[0]) + \
            (a3[0] - a2[0])*(a1[1] - a3[1]))
    gamma = 1.0 - alpha - beta

    # If each barycentric coordinate is greater than 0, the point is inside
    if alpha > 0.0 and beta > 0.0 and gamma > 0.0:
        # If we want the point outside
        if outside:
            # Repeat the procedure
            return point_generator(outside)
        else:
            # Return our random point inside the triangle
            return rand_point
    else: # The point is outside
        if outside:
            # Return our random point outside the triangle
            return rand_point
        else:
            # Repeat the procedure
            return point_generator()
        
# Construct our random point
p = point_generator()

# The inequality values of our constraints in the SDP problem
b = np.array([1,1,2, np.linalg.norm(p - a1), \
     np.linalg.norm(p - a2),                 \
     np.linalg.norm(p - a3)])

#Starting point X
X_temp = np.random.rand(3,3) * 3
X = np.matmul(X_temp.transpose(), X_temp)

#The constraints of our SDP problem
A_ = [np.outer(np.array([1, 0, 0]), np.array([1, 0, 0])), \
     np.outer(np.array([0, 1, 0]), np.array([0, 1, 0])),  \
     np.outer(np.array([1, 1, 0]), np.array([1, 1, 0])),  \
     np.outer(np.append(a1, -1), np.append(a1, -1)),      \
     np.outer(np.append(a2, -1), np.append(a2, -1)),      \
     np.outer(np.append(a3, -1), np.append(a3, -1))]

In [12]:
#The objective barrier function
def phi(a, x, b, mu):
    return (0.5 * np.dot((A_X(a, x) - b).transpose(), A_X(a, x) - b)) \
            - mu * np.log(np.linalg.det(x))

#The gradient of the objective barrier function in 8b
def dphi(a, x, b, mu):
    y = A_X(a, x) - b

    gradf = np.zeros((3, 3))
    for ind, A in enumerate(a):
        gradf = gradf + y[ind] * A
    return gradf - mu*np.linalg.pinv(x)

#The updated gradient of the object barrier function in 8c
def ddphi(a, x, b, mu):
    y = A_X(a, x) - b
    gradf = np.zeros((3, 3))
    for ind, A in enumerate(a):
        gradf = gradf + y[ind] * A
    return np.matmul(np.matmul(x, gradf), x) - mu*x

#Performs the operation AX as enumerated in the assignment
def A_X(a, x):
    element_wise = [np.multiply(A, x) for A in a]
    return np.array([A.sum() for A in element_wise])

In [13]:
#Performs steepest descent
def steepest_descent(op, dop, a, xin, b, mu, niter):
    x = np.copy(xin)
    for i in range(0, niter):
        grad = dop(a,x,b,mu)
        alpha = 1.
        
        #Here, we perform the backtracking line search
        #but we also want the smallest eigenvalue of x^(k+1)
        #to be at least half of the smallest eigenvalue of
        #x^k
        e_val_new, t = np.linalg.eig(x - alpha*grad)
        e_val_old, t = np.linalg.eig(x)
        ratio = np.amin(e_val_new)/np.amin(e_val_old)
        
        #Perform backtracking
        while((phi(a,x - alpha*grad, b, mu) > phi(a, x, b, mu)) \
              or ratio < 0.5):
            alpha *= 0.8
            e_val_new, t = np.linalg.eig(x - alpha*grad)
            e_val_old, t = np.linalg.eig(x)
            ratio = np.amin(e_val_new)/np.amin(e_val_old)
        x -= alpha * grad
        
    return x

In [24]:
x_new = steepest_descent(phi, dphi, A_, X, b, 0, 300);
print "The point value is: ", p
print "The value of optimal solution is", x_new[2, 0:2]

The point value is:  [-0.14318903  1.07715705]
The value of optimal solution is [ 10.67273789  12.39781623]


We note that the recovered solution is incorrect. This is because our solution has no barrier $\mu$ and we are using the ill-conditioned form of gradient descent. Furthermore, the convergence is very slow from the multiple steps that the backtracking search performs (since we have that $\mu = 0$. We try the gradient descent update again with the new gradient:

In [25]:
x_new = steepest_descent(phi, ddphi, A_, X, b, 0, 300);
print "The point value recovered is: ", p
print "The value of optimal solution is", x_new[2, 0:2]

The point value recovered is:  [-0.14318903  1.07715705]
The value of optimal solution is [-0.03817452  0.6715733 ]


Although we do not recover the exact position correctly, we are in the neighborhood (at least in the convex hull) of the correct solution with the correct magnitudes. Although our convergence is much slower, we move close to the expected value. Now, we try new value of $\mu$.

In [26]:
x_new = steepest_descent(phi, dphi, A_, X, b, 1e-4, 300);
print "The point value is: ", p
print "The value of optimal solution is", x_new[2, 0:2]

The point value is:  [-0.14318903  1.07715705]
The value of optimal solution is [  6.5728207   10.20246336]


Although the recovered solution is much better than the previous attempt with $\mu = 0$, we still note that we are still quite off from the correct solution. This is because we are penalizing getting close to 0. However, now the execution time is much quicker than before since we do not have to perform as much backtracking.

In [27]:
x_new = steepest_descent(phi, ddphi, A_, X, b, 1e-4, 300);
print "The point value is: ", p
print "The value of optimal solution is", x_new[2, 0:2]

The point value is:  [-0.14318903  1.07715705]
The value of optimal solution is [-0.03826969  0.67134881]


Again, our solution is not quite what we desire, but we see that the execution time is quicker with the new value of $\mu$ and we recover a solution that is relatively close. This is the best method out of the four methods of gradient descent.